In [1]:
import tensorflow as tf
import keras
import numpy as np
import IPython.display as display
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import datetime, os
from tensorflow.keras.layers import *
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.optimizers import Adam
from IPython.display import clear_output
from sklearn.model_selection import train_test_split
import keras
from tensorflow.keras.applications.densenet import DenseNet201
import cv2
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
import tensorflow as tf
from tensorflow import keras
import segmentation_models as sm
from tensorflow.keras.optimizers import SGD

Segmentation Models: using `keras` framework.


In [2]:
x = np.load("/home/lusk.c/XN_project/images_segmentation.npy")
y = np.load("/home/lusk.c/XN_project/masks_segmentation.npy")
x = x/255
x = np.expand_dims(x, axis = 3)
y = np.expand_dims(y, axis = 3)


In [3]:
n = 0
x2 = np.zeros((x.shape[0],512,512,3), dtype='float32')
while n < x.shape[0]:
    x2[n] = cv2.merge((x[n],x[n],x[n]))
    n +=1
x = 0

In [4]:
n = 0
y2 = np.zeros((y.shape[0],512,512,3), dtype='float32')
while n < y.shape[0]:
    y2[n] = cv2.merge((y[n],y[n],y[n]))
    n +=1
y = 0

In [5]:
from sklearn.model_selection import train_test_split
train_X, val_X,train_y, val_y = train_test_split(x2,y2,test_size=0.2, random_state=0)

In [6]:
# Image size that we are going to use
IMG_SIZE = 512
# Our images are RGB (3 channels)
N_CHANNELS = 1
# Scene Parsing has 150 classes + `not labeled`
N_CLASSES = 2

In [7]:
 base = keras.applications.DenseNet121(input_shape=[512,512,3], 
                                       include_top=False, 
                                       weights='imagenet') 

In [8]:
 skip_names = ['conv1/relu', # size 64*64
              'pool2_relu',  # size 32*32
              'pool3_relu',  # size 16*16
              'pool4_relu',  # size 8*8
              'relu'        # size 4*4
              ] 

In [9]:
 skip_outputs = [base.get_layer(name).output for name in skip_names]
 for i in range(len(skip_outputs)):
     print(skip_outputs[i]) 

KerasTensor(type_spec=TensorSpec(shape=(None, 256, 256, 64), dtype=tf.float32, name=None), name='conv1/relu/Relu:0', description="created by layer 'conv1/relu'")
KerasTensor(type_spec=TensorSpec(shape=(None, 128, 128, 256), dtype=tf.float32, name=None), name='pool2_relu/Relu:0', description="created by layer 'pool2_relu'")
KerasTensor(type_spec=TensorSpec(shape=(None, 64, 64, 512), dtype=tf.float32, name=None), name='pool3_relu/Relu:0', description="created by layer 'pool3_relu'")
KerasTensor(type_spec=TensorSpec(shape=(None, 32, 32, 1024), dtype=tf.float32, name=None), name='pool4_relu/Relu:0', description="created by layer 'pool4_relu'")
KerasTensor(type_spec=TensorSpec(shape=(None, 16, 16, 1024), dtype=tf.float32, name=None), name='relu/Relu:0', description="created by layer 'relu'")


In [10]:
 downstack = keras.Model(inputs=base.input,
                        outputs=skip_outputs)
 downstack.trainable = False 

In [11]:

from tensorflow_examples.models.pix2pix import pix2pix
 # Four upstack blocks for upsampling sizes 
 # 4->8, 8->16, 16->32, 32->64 
upstack = [pix2pix.upsample(512,3),
           pix2pix.upsample(256,3),
           pix2pix.upsample(128,3),
           pix2pix.upsample(64,3)] 

In [12]:
# define the input layer
inputs = keras.layers.Input(shape=[512,512,3])
 # downsample 
down = downstack(inputs)
out = down[-1]
# prepare skip-connections
skips = reversed(down[:-1])
# choose the last layer at first 4 --> 8
# upsample with skip-connections
for up, skip in zip(upstack,skips):
    out = up(out)
    out = keras.layers.Concatenate()([out,skip])
# define the final transpose conv layer
# image 128 by 128 with 59 classes
out = keras.layers.Conv2DTranspose(3, 3,
                                   strides=2,
                                   padding='same',
                                   name=None,
                                   )(out)
 # complete unet model
unet = keras.Model(inputs=inputs, outputs=out) 

In [ ]:
train_X = tf.data.Dataset.from_tensor_slices(train_X)
val_X = tf.data.Dataset.from_tensor_slices(val_X)
train_y = tf.data.Dataset.from_tensor_slices(train_y)
val_y = tf.data.Dataset.from_tensor_slices(val_y)
train_X.element_spec, train_y.element_spec, val_X.element_spec, val_y.element_spec 

In [ ]:
train = tf.data.Dataset.zip((train_X, train_y))
val = tf.data.Dataset.zip((val_X, val_y)) 

In [ ]:
 def brightness(img, mask):
    img = tf.image.adjust_brightness(img, 0.1)
    return img, mask
 
def gamma(img, mask):
    img = tf.image.adjust_gamma(img, 0.1)
    return img, mask
def hue(img, mask):
     img = tf.image.adjust_hue(img, -0.1)
     return img, mask

def crop(img, mask):
    img = tf.image.central_crop(img, 0.7)
    img = tf.image.resize(img, (128,128))
    mask = tf.image.central_crop(mask, 0.7)
    mask = tf.image.resize(mask, (128,128))
    mask = tf.cast(mask, tf.float32)
    return img, mask

def flip_hori(img, mask):
    img = tf.image.flip_left_right(img)
    mask = tf.image.flip_left_right(mask)
    return img, mask

def flip_vert(img, mask):
    img = tf.image.flip_up_down(img)
    mask = tf.image.flip_up_down(mask)
    return img, mask

def rotate(img, mask):
    img = tf.image.rot90(img)
    mask = tf.image.rot90(mask)
    return img, mask 

In [ ]:
 # perform augmentation on train data only
b = train.map(flip_hori)

train = train.concatenate(b)

In [ ]:
BATCH = 64
AT = tf.data.AUTOTUNE
BUFFER = 1000
STEPS_PER_EPOCH = 800//BATCH
VALIDATION_STEPS = 200//BATCH
train = train.cache().shuffle(BUFFER).batch(BATCH).repeat()
train = train.prefetch(buffer_size=AT)
val = val.batch(BATCH) 

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import layers, callbacks
from keras.callbacks import LearningRateScheduler
from tensorflow.keras import backend as K
from tensorflow.keras.losses import binary_crossentropy
def dice_coeff(y_true, y_pred):
    smooth = 1.
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    score = (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)
    return score


def dice_loss(y_true, y_pred):
    loss = 1 - dice_coeff(y_true, y_pred)
    return loss

def bce_dice_loss(y_true, y_pred):
    loss = binary_crossentropy(y_true, y_pred) + dice_loss(y_true, y_pred)
    return loss
def lr_scheduler(epoch):
    if epoch < 22:
        return 0.1
    elif epoch < 44:
        return .01
    elif epoch < 80:
        return 0.001
    else:
        return .000001
scheduler = LearningRateScheduler(lr_scheduler)


early_stopping_cb = keras.callbacks.EarlyStopping(
    min_delta=0.001, # minimium amount of change to count as an improvement
    patience=10, # how many epochs to wait before stopping
)



In [ ]:
unet.compile(loss= bce_dice_loss,
             optimizer=keras.optimizers.RMSprop(learning_rate=0.1),
             metrics=dice_coeff) 


In [ ]:
hist = unet.fit(train,
                validation_data=val,
                steps_per_epoch=STEPS_PER_EPOCH,
                validation_steps=VALIDATION_STEPS,
                epochs=1000) 

In [ ]:
img, mask = next(iter(val))
pred = unet.predict(img)

plt.figure(figsize=(10,5))
for i in pred:
    plt.subplot(121)
    i = tf.argmax(i, axis=-1)
    plt.imshow(i,cmap='jet')
    plt.axis('off')
    plt.title('Prediction')
    break

plt.subplot(122)
plt.imshow(mask[0], cmap='jet')
plt.axis('off')
plt.title('Ground Truth')
plt.show() 

In [ ]:
def display(display_list):
    fig = plt.figure(figsize=(15, 15))
    title = ['Input Image', 'True Mask', 'Predicted Mask']

    for i in range(len(display_list)):
        plt.subplot(1, len(display_list), i + 1)
        plt.title(title[i])
        plt.imshow(tf.keras.preprocessing.image.array_to_img
        (display_list[i]))
        plt.axis('off')
    plt.show()

def show_predictions(dataset=None, num=1):
    for image, mask in dataset.take(num):
        pred_mask = unet.predict(image)
        pred_mask *= 255.0
        print(pred_mask.min())
        print(pred_mask.max())
        print(np.unique(pred_mask, return_counts=True))
        display([image[0], mask[0], pred_mask[0]])

show_predictions(val, 3)